<a href="https://colab.research.google.com/github/Nicolenki7/Customer-Churn-Project/blob/main/Customer_Churn_Python%2BSQlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Importar la librería Pandas
import pandas as pd

# 2. Cargar el dataset
# Asegúrate de que el archivo 'WA_Fn-UseC_-Telco-Customer-Churn.csv' esté en el mismo directorio.
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# 3. Inspección inicial (mostrar las primeras 5 filas y la información de tipos de datos)
print("--- Primeras Filas del Dataset ---")
print(df.head())
print("\n--- Información de Tipos de Datos (Dtypes) ---")
df.info()

--- Primeras Filas del Dataset ---
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV S

In [2]:
# Convertir la columna 'TotalCharges' a tipo numérico (float)
# Los valores no numéricos se convertirán a NaN.
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Verificar cuántos valores NaN se generaron (estos son los clientes nuevos con total=0)
print("\n--- Conteo de Valores Faltantes (NaN) en TotalCharges ---")
print(df['TotalCharges'].isnull().sum())


--- Conteo de Valores Faltantes (NaN) en TotalCharges ---
11


In [3]:
# Reemplazar los valores NaN en 'TotalCharges' por 0.0
# Esto asume que los clientes con NaN son clientes nuevos que aún no han pagado un cargo total.
df['TotalCharges'].fillna(0, inplace=True)

# Verificar que ya no hay valores faltantes en 'TotalCharges'
print("\n--- Verificación de Valores Faltantes después de la Imputación ---")
print(df['TotalCharges'].isnull().sum())


--- Verificación de Valores Faltantes después de la Imputación ---
0


/tmp/ipython-input-3477206730.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(0, inplace=True)


In [4]:
# Codificar la variable objetivo 'Churn': 'Yes' -> 1, 'No' -> 0
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})

# Opcional: Codificar otras variables binarias (Sí/No) de alto impacto
# Se pueden agrupar en una lista y aplicar la misma transformación:
bin_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

for col in bin_cols:
    df[col] = df[col].replace({'Yes': 1, 'No': 0})

print("\n--- Valores Únicos en Columna Churn (Verificación) ---")
print(df['Churn'].unique())


--- Valores Únicos en Columna Churn (Verificación) ---
[0 1]


/tmp/ipython-input-3059931602.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})
/tmp/ipython-input-3059931602.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'Yes': 1, 'No': 0})


In [5]:
# Columnas que contienen el valor 'No internet service'
internet_service_cols = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Reemplazar 'No internet service' por 'No'
# Esto hace que la categoría 'No' incluya tanto a los que no tienen el servicio específico
# como a los que no tienen el servicio de Internet en absoluto.
for col in internet_service_cols:
    df[col] = df[col].replace('No internet service', 'No')

# Verificar los valores únicos en una columna ejemplo (deberían ser solo 'Yes' o 'No')
print("\n--- Valores Únicos Consolidados en OnlineSecurity ---")
print(df['OnlineSecurity'].unique())


--- Valores Únicos Consolidados en OnlineSecurity ---
['No' 'Yes']


In [6]:
# Eliminar la columna 'customerID' ya que no es necesaria para el análisis o el modelo
df.drop('customerID', axis=1, inplace=True)

print("\n--- Columnas Restantes después de Eliminar customerID ---")
print(df.columns.tolist())


--- Columnas Restantes después de Eliminar customerID ---
['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']


In [7]:
# Guardar el DataFrame df (limpio y listo) en un nuevo archivo CSV.
# Esto asegura que los cambios realizados en Python se mantengan.

# 1. Definir el nombre del archivo de salida
output_file = 'telco_churn_limpio.csv'

# 2. Exportar el DataFrame a CSV
# index=False es crucial para evitar que la columna de índice de Pandas se guarde como una nueva columna.
df.to_csv(output_file, index=False)

print(f"\n--- Exportación Exitosa ---")
print(f"El DataFrame limpio se ha guardado como: {output_file}")
print("¡El archivo está listo para ser importado en SQL!")


--- Exportación Exitosa ---
El DataFrame limpio se ha guardado como: telco_churn_limpio.csv
¡El archivo está listo para ser importado en SQL!


In [8]:
import sqlite3

# 1. Conectar a una base de datos SQLite (si no existe, la crea)
conn = sqlite3.connect('telco_churn_db.sqlite')
cursor = conn.cursor()

# 2. Cargar el DataFrame limpio en una tabla de SQLite
# name='churn_data' será el nombre de la tabla en tu base de datos
df.to_sql('churn_data', conn, if_exists='replace', index=False)

print("\n--- Carga en SQLite Exitosa ---")
print("Tabla 'churn_data' creada.")


--- Carga en SQLite Exitosa ---
Tabla 'churn_data' creada.


In [9]:
# Consulta SQL Estratégica para el Feature Engineering
sql_query = """
SELECT
    *,
    CASE
        WHEN tenure <= 12 THEN '0-1 Year'
        WHEN tenure > 12 AND tenure <= 36 THEN '1-3 Years'
        WHEN tenure > 36 AND tenure <= 60 THEN '3-5 Years'
        ELSE '5+ Years'
    END AS Seniority_Category,
    CASE
        WHEN MonthlyCharges < 30 THEN 'Low'
        WHEN MonthlyCharges >= 30 AND MonthlyCharges < 80 THEN 'Medium'
        ELSE 'High'
    END AS Monthly_Charges_Level
FROM
    churn_data;
"""

# Ejecutar la consulta y guardar el resultado en un nuevo DataFrame de Pandas
df_fe = pd.read_sql_query(sql_query, conn)

# 3. Cerrar la conexión a la base de datos
conn.close()

print("\n--- Feature Engineering en SQL Exitoso ---")
print("Nuevas columnas generadas:")
print(df_fe[['tenure', 'Seniority_Category', 'MonthlyCharges', 'Monthly_Charges_Level']].head())


--- Feature Engineering en SQL Exitoso ---
Nuevas columnas generadas:
   tenure Seniority_Category  MonthlyCharges Monthly_Charges_Level
0       1           0-1 Year           29.85                   Low
1      34          1-3 Years           56.95                Medium
2       2           0-1 Year           53.85                Medium
3      45          3-5 Years           42.30                Medium
4       2           0-1 Year           70.70                Medium


In [10]:
# 1. Importar la función de descarga de Google Colab
from google.colab import files

# 2. Definir el nombre del archivo final
# Es crucial usar un nombre descriptivo para tu portafolio
FINAL_CSV_FILE = 'telco_churn_feature_engineered_for_tableau.csv'

# 3. Guardar el DataFrame final (df_fe) como un archivo CSV
# Asegúrate de usar 'df_fe', que es el DataFrame que contiene las nuevas columnas de SQL
# index=False es fundamental para no añadir una columna de índice innecesaria
df_fe.to_csv(FINAL_CSV_FILE, index=False)

# 4. Forzar la descarga del archivo a tu sistema local
# Esto abrirá una ventana de descarga en tu navegador
files.download(FINAL_CSV_FILE)

print(f"\n--- Descarga Iniciada ---")
print(f"El archivo '{FINAL_CSV_FILE}' se ha guardado y la descarga a tu PC ha comenzado.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


--- Descarga Iniciada ---
El archivo 'telco_churn_feature_engineered_for_tableau.csv' se ha guardado y la descarga a tu PC ha comenzado.
